In [1]:
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np
import datetime
from datetime import date
import time
from urllib.request import Request, urlopen

import urllib.request
from bs4 import BeautifulSoup
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

credentials = {'CREDENTIALS_HERE'}
gc = gspread.service_account_from_dict(credentials)

In [85]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1VEl13dz9nGJVkKeNWUOZfimtSHKxg0owkLX-rivMpJc/edit#gid=0')
sheet = wb.worksheet('Sheet1')

schools = sheet.get_all_values()

In [90]:
new_schools = []
for i in range(len(schools)):
    new_schools.append(schools[i][0])
new_schools.remove("")

In [15]:
url = "https://datausa.io/profile/university/college-of-the-siskiyous"
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

req = urllib.request.Request(url, headers=hdr)
response = urllib.request.urlopen(req).read()
soup = BeautifulSoup(response)

In [99]:
for n in range(len(new_schools)):
    college_name = new_schools[n]
    url = "https://datausa.io/profile/university/"+ college_name.replace(' ', '-').lower()

    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}
    
    try:
        req = urllib.request.Request(url, headers=hdr)
        response = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(response)

        all_stats = BeautifulSoup(str(soup.find_all('div', class_="content-container")[-1]))
        all_stats = all_stats.find_all('div', class_="Stat large-text")

        stats_df = pd.DataFrame(columns=['School', 'Link', 'Total Students'])

        for i in range(len(all_stats)):
            important_stat = all_stats[i]
            # print(important_stat)
            stat_title = BeautifulSoup(str(important_stat)).find('div',"stat-title").text

            if stat_title.find('Enrolled Students') != -1:
                stat_value = BeautifulSoup(str(important_stat)).find('div',"stat-value").text
                # print(stat_value)
                
                stats_df = stats_df.append({'School': college_name, 'Link':url, 'Total Students':stat_value}, ignore_index=True)
    except:
        continue
        


In [100]:
stats_df

,School,Link,Total Students
0,Yuba College,https://datausa.io/profile/university/yuba-col...,"5,626"


In [56]:
all_stats

<html><body><div class="content-container"><div class="profile-stats"><div class="Stat large-text"><div class="stat-title">2019 Undergraduate Tuition</div><div class="stat-value">$1,104</div><div class="stat-subtitle">0% growth from 2018</div></div><div class="Stat large-text"><div class="stat-title">2019 Average Net Price</div><div class="stat-value">$9,073</div><div class="stat-subtitle">After Financial Aid</div></div><div class="Stat large-text"><div class="stat-title">2017 Student Loan</div><div class="stat-value">20.8%</div><div class="stat-subtitle">Default Rate</div></div><div class="Stat large-text"><div class="stat-title">2019 Enrolled Students</div><div class="stat-value">1,761</div><div class="stat-subtitle">44% Full-Time</div></div><div class="Stat large-text"><div class="stat-title">2018 Graduation Rate</div><div class="stat-value">24%</div><div class="stat-subtitle">35 Graduates</div></div></div></div></body></html>

In [52]:
stat_title

'2019 Undergraduate Tuition'

In [53]:
important_stats

<div class="Stat large-text"><div class="stat-title">2019 Undergraduate Tuition</div><div class="stat-value">$1,104</div><div class="stat-subtitle">0% growth from 2018</div></div>

In [54]:
stats_df

,School,Link,Stat Title,Stat Value
